In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.decomposition import PCA
import warnings
import os
import matplotlib.pyplot as plt
import seaborn as sns  # Used for plotting the cross-tabulation
import dask  # Added dask import

warnings.filterwarnings('ignore')
# Suppress specific warnings
warnings.filterwarnings('ignore', category=FutureWarning)
# Configure dask (assuming it is installed)
# This configuration is often used to optimize dask/pandas interactions
try:
    import dask.config
    dask.config.set({"dataframe.query-planning": True})
except ImportError:
    # If dask is not installed, the configuration is skipped.
    pass


In [2]:

# --- VISUALIZATION SETTINGS (WHITE THEME) ---
# 1. Enforce White Background for Seaborn
sns.set_style("white")
sns.set_context("paper", font_scale=1.2)

# 2. Enforce White Background for Matplotlib (Global)
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['savefig.facecolor'] = 'white'

# 3. Configure Scanpy
sc.set_figure_params(dpi=150, figsize=(8, 8), facecolor='white')

# CONFIGURATION
INPUT_FILE = "../../data/Kidney_ST/GSE211785_7_13_23_slide0_annotated_iPTsubclusters.h5ad"
# Using the absolute path provided by the user for the output file
OUTPUT_FILE = "../../data/Kidney_ST/Kidney_CosMx_BANKSY_Calculated.h5ad"
LAMBDA_PARAM = 0.2  # The BANKSY "mixing" parameter
K_NEIGHBORS = 15    # Number of spatial neighbors
RESOLUTION = 0.5    # Resolution for Leiden clustering
DOMAIN_KEY = 'BANKSY_Domain'
OUTPUT_DIR = '../../analysis/domainanalysis'


In [3]:

# --- Utility Function to Save Plots ---


def save_plots(fig, filename):
    """Saves the figure in both PNG and PDF formats."""
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    png_path = os.path.join(OUTPUT_DIR, f"{filename}.png")
    pdf_path = os.path.join(OUTPUT_DIR, f"{filename}.pdf")

    print(f"Saving PNG to: {png_path}")
    fig.savefig(png_path, dpi=300, bbox_inches='tight')
    print(f"Saving PDF to: {pdf_path}")
    fig.savefig(pdf_path, bbox_inches='tight')
    plt.close(fig)


In [4]:

# --- BANKSY Core Algorithm ---


def run_banksy_jointly(adata, k_geom, lambda_val):
    """
    Runs BANKSY on the entire dataset.
    """
    print("--- STARTING BANKSY ALGORITHM ---")

    # A. PREPARE EXPRESSION MATRIX (M_cell)
    if sp.issparse(adata.X):
        current_max = adata.X.max()
    else:
        current_max = np.max(adata.X)

    # Check for normalization/logarithmization based on max expression
    if 'log1p' not in adata.uns and current_max > 50:
        print("Normalizing and logging data...")
        sc.pp.normalize_total(adata, target_sum=1e4)
        sc.pp.log1p(adata)

    X_cell = adata.X.copy()
    if sp.issparse(X_cell):
        X_cell = X_cell.toarray()

    print(f"Expression Matrix Shape: {X_cell.shape}")

    # B. COMPUTE SPATIAL NEIGHBORS (The Graph)
    print(f"Computing spatial graph (k={k_geom})...")
    # Ensure to use the standard 'spatial' coordinates
    sc.pp.neighbors(adata, n_neighbors=k_geom, use_rep='spatial')
    spatial_conn = adata.obsp['connectivities']

    # C. COMPUTE NEIGHBORHOOD MATRIX (M_neighbor)
    print("Computing neighborhood transcriptome...")

    # Row-normalize spatial connectivity matrix to get AVERAGE expression
    row_sums = np.array(spatial_conn.sum(1)).flatten()
    row_sums[row_sums == 0] = 1  # Avoid division by zero
    deg_inv = sp.diags(1.0 / row_sums)
    avg_conn = deg_inv.dot(spatial_conn)
    X_neighbor = avg_conn.dot(X_cell)

    # D. CONSTRUCT BANKSY MATRIX (M_star)
    w_cell = np.sqrt(1 - lambda_val)
    w_neighbor = np.sqrt(lambda_val)

    print(f"Constructing Augmented Matrix (Lambda={lambda_val})...")
    M_star = np.hstack([w_cell * X_cell, w_neighbor * X_neighbor])

    print(f"BANKSY Matrix Shape: {M_star.shape}")

    # E. CLUSTERING
    print("Clustering BANKSY domains...")
    adata_banksy = sc.AnnData(X=M_star)

    # Run standard clustering pipeline on this new matrix
    sc.tl.pca(adata_banksy, svd_solver='arpack', n_comps=50)
    sc.pp.neighbors(adata_banksy, n_neighbors=15, n_pcs=50)
    sc.tl.leiden(adata_banksy, resolution=RESOLUTION, key_added=DOMAIN_KEY)

    # F. SAVE RESULTS TO ORIGINAL OBJECT
    print("Transferring results back to main object...")
    adata.obs[DOMAIN_KEY] = adata_banksy.obs[DOMAIN_KEY].values

    # Compute UMAP of BANKSY embedding for visualization
    sc.tl.umap(adata_banksy)
    adata.obsm['X_umap_BANKSY'] = adata_banksy.obsm['X_umap']

    print(f"Saving results to: {OUTPUT_FILE}")
    adata.write(OUTPUT_FILE)

    return adata



In [5]:

# %%
# --- MAIN EXECUTION LOGIC ---

# 1. CONDITIONAL LOAD
adata = None
calculation_needed = True

if os.path.exists(OUTPUT_FILE):
    try:
        # Load minimally to check if BANKSY_Domain is already present in the output file
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            # Using backed='r' allows a quick check without loading the whole matrix
            temp_adata = sc.read_h5ad(OUTPUT_FILE, backed='r')

            if DOMAIN_KEY in temp_adata.obs:
                print(
                    f"Found existing BANKSY analysis in {OUTPUT_FILE}. Loading this file and skipping calculation.")
                adata = sc.read_h5ad(OUTPUT_FILE)
                calculation_needed = False

            # Close the temporary backed object
            del temp_adata

    except Exception as e:
        # If the file exists but is corrupted or incompatible, we fall back to recalculation
        print(
            f"Warning: Could not minimally read {OUTPUT_FILE} or check for {DOMAIN_KEY}. Error: {e}")

if calculation_needed:
    print(
        f"BANKSY results not found or incomplete. Loading raw data from {INPUT_FILE} for calculation.")
    adata = sc.read_h5ad(INPUT_FILE)

# Ensure spatial coords are standard (fixes potential pandas/numpy issue)
if hasattr(adata.obsm['spatial'], 'to_numpy'):
    adata.obsm['spatial'] = adata.obsm['spatial'].to_numpy()

# 2. CONDITIONAL EXECUTION
if calculation_needed:
    print(
        f"\n--- Output file or '{DOMAIN_KEY}' not found. RUNNING BANKSY calculation. ---")
    adata = run_banksy_jointly(
        adata, k_geom=K_NEIGHBORS, lambda_val=LAMBDA_PARAM)
else:
    print(
        f"\n--- Output file already loaded and '{DOMAIN_KEY}' column found. SKIPPING BANKSY calculation. ---")



Found existing BANKSY analysis in ../../data/Kidney_ST/Kidney_CosMx_BANKSY_Calculated.h5ad. Loading this file and skipping calculation.

--- Output file already loaded and 'BANKSY_Domain' column found. SKIPPING BANKSY calculation. ---


In [6]:

# 3. VISUALIZATION AND PLOTTING
print("\n--- STARTING VISUALIZATION ---")

# A. Spatial Plot of the new BANKSY Domains
fig1 = sc.pl.spatial(
    adata,
    color=DOMAIN_KEY,
    title=f"Spatial Domains (BANKSY $\\lambda$={LAMBDA_PARAM})",
    spot_size=0.01,
    show=False,
    return_fig=True
)
save_plots(fig1, '1_banksy_spatial_domains')



--- STARTING VISUALIZATION ---


Saving PNG to: ../../analysis/domainanalysis/1_banksy_spatial_domains.png
Saving PDF to: ../../analysis/domainanalysis/1_banksy_spatial_domains.pdf


In [7]:

# B. UMAP Plot of the BANKSY Embedding
# FIX: Use sc.pl.embedding instead of sc.pl.umap to plot custom embeddings (like X_umap_BANKSY)
if 'X_umap_BANKSY' in adata.obsm:
    fig2 = sc.pl.embedding(
        adata,
        basis='X_umap_BANKSY',
        color=DOMAIN_KEY,
        title=f"UMAP of BANKSY Embedding ({DOMAIN_KEY})",
        show=False,
        return_fig=True
    )
    save_plots(fig2, '2_banksy_umap_embedding')
else:
    print("Warning: X_umap_BANKSY not found. Skipping UMAP plot.")



Saving PNG to: ../../analysis/domainanalysis/2_banksy_umap_embedding.png
Saving PDF to: ../../analysis/domainanalysis/2_banksy_umap_embedding.pdf


In [8]:

# C. Spatial Plot colored by existing 'type' annotation
if 'type' in adata.obs:
    fig3 = sc.pl.spatial(
        adata,
        color='type',
        title="Spatial Condition type",
        spot_size=0.01,
        show=False,
        return_fig=True
    )
    save_plots(fig3, '3_original_spatial_types')
else:
    print("Warning: 'type' column not found for comparison plot.")


Saving PNG to: ../../analysis/domainanalysis/3_original_spatial_types.png
Saving PDF to: ../../analysis/domainanalysis/3_original_spatial_types.pdf


In [11]:

# E. Cross-Tabulation (Domain Distribution)
if 'type' in adata.obs:
    print("\nDomain vs. Cell Type Distribution:")
    ct = pd.crosstab(adata.obs[DOMAIN_KEY], adata.obs['type'])
    print(ct)

    # F. Plotting the Cross-Tabulation (Now fig5)

    # Normalize the cross-tabulation table across each domain (row sums to 1)
    ct_norm = ct.div(ct.sum(axis=1), axis=0)

    fig5, ax = plt.subplots(figsize=(12, 6))  # Note: changed to fig5

    # The global plt.rcParams settings applied earlier handle the white background.

    # Create a stacked bar plot
    # Trigger image search for a relevant visualization

    ct_norm.plot(kind='bar', stacked=True, ax=ax)

    ax.set_title(f'Cell Type Composition within BANKSY Domains', fontsize=16)
    ax.set_xlabel(f'BANKSY Domain ({DOMAIN_KEY})', fontsize=14)
    ax.set_ylabel('Proportion of Cells', fontsize=14)
    ax.set_xticklabels(ct_norm.index, rotation=45, ha='right')

    # Move the legend outside the plot
    ax.legend(title='Original Cell Type',
              bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()

    # Note: changed filename index
    save_plots(fig5, '5_domain_vs_type_stacked_bar_plot')

    # G. Save the cross-tabulation table as a markdown file (Now index 6)
    markdown_table = ct.to_markdown()

    with open(os.path.join(OUTPUT_DIR, '6_domain_vs_type_crosstab.md'), 'w') as f:
        f.write(
            f"## Cross-Tabulation of {DOMAIN_KEY} vs. Original Cell Type\n\n")
        f.write(markdown_table)
    print(
        f"Cross-tabulation saved to {OUTPUT_DIR}/6_domain_vs_type_crosstab.md")


# E. Cross-Tabulation (Domain Distribution)
if 'cellType_CosMx_2' in adata.obs:
    print("\nDomain vs. Cell Type Distribution:")
    ct = pd.crosstab(adata.obs[DOMAIN_KEY], adata.obs['cellType_CosMx_2'])
    print(ct)

    # F. Plotting the Cross-Tabulation (Now fig5)

    # Normalize the cross-tabulation table across each domain (row sums to 1)
    ct_norm = ct.div(ct.sum(axis=1), axis=0)

    fig5, ax = plt.subplots(figsize=(12, 6))  # Note: changed to fig5

    # The global plt.rcParams settings applied earlier handle the white background.

    # Create a stacked bar plot
    # Trigger image search for a relevant visualization

    ct_norm.plot(kind='bar', stacked=True, ax=ax)

    ax.set_title(f'Cell Type Composition within BANKSY Domains', fontsize=16)
    ax.set_xlabel(f'BANKSY Domain ({DOMAIN_KEY})', fontsize=14)
    ax.set_ylabel('Proportion of Cells', fontsize=14)
    ax.set_xticklabels(ct_norm.index, rotation=45, ha='right')

    # Move the legend outside the plot
    ax.legend(title='Original Cell Type',
              bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()

    # Note: changed filename index
    save_plots(fig5, '6_domain_vs_cell_type_stacked_bar_plot')

    # G. Save the cross-tabulation table as a markdown file (Now index 6)
    markdown_table = ct.to_markdown()

    with open(os.path.join(OUTPUT_DIR, '7_domain_vs_cell_type_crosstab.md'), 'w') as f:
        f.write(
            f"## Cross-Tabulation of {DOMAIN_KEY} vs. Original Cell Type\n\n")
        f.write(markdown_table)
    print(
        f"Cross-tabulation saved to {OUTPUT_DIR}/6_domain_vs_type_crosstab.md")


print("\n--- ALL DONE ---")
print(f"Results saved to {OUTPUT_FILE}")
print(f"Plots saved in {OUTPUT_DIR}")



Domain vs. Cell Type Distribution:
type           Disease  Healthy
BANKSY_Domain                  
0                58951    35281
1                27821    50580
2                43527    26425
3                40639     9504
4                16243    33607
5                15441    27675
6                23030    18191
7                16785    22146
8                 4370    21396
9                 1360    20288
10               15283     1440
11                7423     9277
12                1611     7238
13                2232      150
14                 244      443


Saving PNG to: ../../analysis/domainanalysis/5_domain_vs_type_stacked_bar_plot.png
Saving PDF to: ../../analysis/domainanalysis/5_domain_vs_type_stacked_bar_plot.pdf
Cross-tabulation saved to ../../analysis/domainanalysis/6_domain_vs_type_crosstab.md

Domain vs. Cell Type Distribution:
cellType_CosMx_2    DCT  Endothelium  Fibro  IC/DCT  Immune  Injured TAL  \
BANKSY_Domain                                                              
0                   268        32755  23585    1433   23758         3158   
1                    30          735    378      79     545           71   
2                    15           32     52      51      41          558   
3                   116         1566  37893     235    4258         1315   
4                     1            8     43       0       7          173   
5                    82         3988   4629     135    3403         1167   
6                   224          897    779   11513     724          685   
7                   178      